## 네이버금융_재무제표_크롤러

'종목코드'를 입력하면 셀레늄으로 재무제표를 크롤링해오는 프로그램

10개 기업만 테스트로 시도해 본 결과 정상 작동함. 

## 조금 손본 후 '네이버금융_최종'으로 따로 저장함. 
### 수정내역 
#### 1)1000개 이상의 기업을 크롤링하므로 창을 하나하나 닫는 코드 추가
#### 2) 상장폐지된 기업, 인수 합병된 기업 존재하기에 try, except로 예외처리함. 

In [1]:
import requests
from bs4 import BeautifulSoup 
from datetime import datetime                            
import pandas as pd 
import time 
import urllib.request 
from selenium.webdriver import Chrome
import json 
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import time

In [2]:
before_list = '''
000040
000080
000100
000000
000120

'''

In [3]:
company_list = before_list.split()

In [4]:
company_list

['000040', '000080', '000100', '000000', '000120']

## 기업의 '종목코드' 리스트화 해서 넣기

밑의 리스트는 예시

## 현재 궁금증, 고민거리
1)time,sleep은 몇 초 정도로 할지? 코스피, 코스닥 상장사 약 2000개 전부를 크롤링한다 생각했을 때 각 time.sleep에서 일초씩 차이가 나는 것은 총 소요시간에서 큰 차이로 이어짐. 

2)비정형 패널데이터 선형회귀 분석은 책을 한두권 사서 더 공부해보고 분석 진행. 이론이 흔들리면 안된다. 

3)왜 굳이 비정형 패널데이터 분석을 하는지? 이에 대한 소명 필요. 

4)유니코드 공백 지우기..?잘 모르겠다.. 



In [5]:
def stock_crawler(code):
    
    browser  = Chrome()
    browser.maximize_window()
    
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    
    browser.get(base_url)
    time.sleep(1.5)
   
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    time.sleep(1.5)
    
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()
    time.sleep(1.5)

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명
    title0 = html1.find('head').find('title').text
    print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date 들고오기
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns 들고오기
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text: #유니코드의 공백을 없애주는 방법.
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    ###
    
    multiple_index = []  
 
    for i in range(len(date)):
      
        multiple_index.append(code)
    
    
    
    
    ####
    df = pd.DataFrame(td2,columns = col,index = [multiple_index, date])
    
    browser.close()
    
    
    ###일단 어떤게 필요할지 몰라 다 들고왔지만 5개년, 즉 5행까지만 쓰는 걸로 잠정적 결론.
    df.drop(df.index[5:], inplace=True)
    time.sleep(1)
    
    
    return df

In [6]:
df_first_row = stock_crawler('000020')

<ipython-input-5-6b58ed1c41a0>:13: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))


 기업개요(동화약품)


In [7]:
df_first_row

매출액 영업이익 영업이익(발표기준) 세전계속사업이익 당기순이익 당기순이익(지배) 당기순이익(비지배)  \
000020 2016/12  2,375  113        113      357   263       263          0   
       2017/12  2,589  110        110      652   470       470          0   
       2018/12  3,066  112        112      145   101       101          0   
       2019/12  3,072   96         96      162    91        91          0   
       2020/12  2,721  232        232      336   287       285          2   

                 자산총계 부채총계   자본총계  ...   부채비율     자본유보율 EPS(원) PER(배)  BPS(원)  \
000020 2016/12  3,246  717  2,529  ...  28.34    802.68    940   8.64   9,143   
       2017/12  3,672  703  2,969  ...  23.67    955.79  1,683   5.79  10,756   
       2018/12  3,706  732  2,974  ...  24.61    967.43    360  25.24  10,761   
       2019/12  3,764  766  2,997  ...  25.57    976.07    326  25.46  10,856   
       2020/12  4,338  911  3,427  ...  26.59  1,064.44  1,021  19.25  11,999   

               PBR(배) 현금DPS(원) 현금배당수익률 현금배당성향(%)  발행주식수(보통주)  
000020 2016/12   0.89      110    1.35     11.59  27,931,470  
       2017/12   0.91      150    1.54      8.81  27,931,470  
       2018/12   0.85      120    1.32     32.90  27,931,470  
       2019/12   0.77      120    1.44     36.34  27,931,470  
       2020/12   1.64      180    0.92     17.43           0  

[5 rows x 33 columns]

# for 문으로 모든 기업 크롤링 진행


In [8]:
for i in company_list :
    
    try : 
        df_new = stock_crawler(i)
        df_first_row = pd.concat([df_first_row, df_new])
        
    except:
        
        time.sleep(1)
        continue

<ipython-input-5-6b58ed1c41a0>:13: DeprecationWarning: use driver.switch_to.frame instead
  browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))


 기업개요(KR모터스)
 기업개요(하이트진로)
 기업개요(유한양행)


NameError: name 'browser' is not defined

In [ ]:
#엑셀로 변환
df_all = df_first_row
df_all.to_excel('tobinq.xlsx')

# 상의할 점

1. 기업들을 어떤 표준에 의거해서 산업 별로 분류할지? 
2. 캡스톤 디자인 예산 신청해서 그 돈으로 선형회귀 관련 책을 구매하는 건 가능할지?